In [ ]:
#IMPORT NECESSARY LIBRARIES
from osgeo import gdal,ogr,osr
import os, sys

In [ ]:
#IMPORT SHAPEFILES
#DEFINE THE DIRECTORY TO THE SHAPEFILE
inshpdir = input('INPUT VECTOR FILE: ')
#GET DRIVER BY NAME
driver = ogr.GetDriverByName('ESRI Shapefile')
#OPEN SHAPEFILE
dataset = driver.Open(inshpdir)
layer = dataset.GetLayer()

In [ ]:
#DEFINE VN2000 (CAN SKIP IF SOURCE DATA IS NOT IN VN2000)
#DEFINE CENTRAL MERIDIAN
centralmerid = input('PLEASE INPUT CENTRAL MERIDIAN OF INPUT FILE: ')
#DEFINE SCALE
scale = input('PLEASE CHOOSE SCALE OF INPUT FILE (0.9996/0.9999): ')
#DEFINE VN2000 PROJECTION
prj = '''PROJCS["VN-2000 {} /", 
    GEOGCS["VN-2000",_
        DATUM["Vietnam_2000",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            TOWGS84[-191.90441429,-39.30318279,-111.45032835,-0.00928836,0.01975479,-0.00427372,1.000000252906278],
            AUTHORITY["EPSG","6756"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4756"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",{}],
    PARAMETER["scale_factor",{}],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","3405"]]'''.format(centralmerid,centralmerid,scale)

In [ ]:
#DEFINE TRANSFORMATION 
vnspatref = osr.SpatialReference()
vnspatref.ImportFromWkt(prj)
wgsspatref = osr.SpatialReference()
wgsspatref.ImportFromEPSG(4326)
transform = osr.CoordinateTransformation(vnspatref,wgsspatref)

In [ ]:
#REPROJECT SHAPEFILES
#DEFINE OUTPUT REPROJECTED SHAPEFILE
newfulname = inshpdir.split('.')[-2] + '_GEOWGS.shp'    
newname = inshpdir.split('.')[-2].split('\\')[-1]
driver = ogr.GetDriverByName('ESRI Shapefile')
outdataset = driver.CreateDataSource(newfulname)
outLayer = outdataset.CreateLayer(newname+'_GEOWGS',geom_type = ogr.wkbPolygon) 

In [ ]:
#GET FIELD DEFINITION FOR OUTPUT SHAPEFILE
inLayerDefn = layer.GetLayerDefn()    
for i in range(0, inLayerDefn.GetFieldCount()):
    fieldDefn = inLayerDefn.GetFieldDefn(i)
    outLayer.CreateField(fieldDefn)
outLayerDefn = outLayer.GetLayerDefn()

In [ ]:
#REPROJECT SHAPEFILES
for feature in layer:    
    geom = feature.GetGeometryRef()        
    geom.Transform(transform)    
    outFeature = ogr.Feature(outLayerDefn)    
    outFeature.SetGeometry(geom)        
    for i in range(0,outLayerDefn.GetFieldCount()):            
        outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), feature.GetField(i))    
    outLayer.CreateFeature(outFeature)    
    outFeature = None          
wgsspatref.MorphToESRI()

In [ ]:
#CREATE PROJECTION FILE (.PRJ)
 newfulnamenoext = newfulname.split('.')[-2]
file = open(newfulnamenoext+'.prj', 'w') 
file.write(wgsspatref.ExportToWkt())
file.close()